In [1]:
import logging
log_level = logging.INFO
# log_level = logging.DEBUG
log_level = 15
logging.basicConfig(level=log_level)
logger = logging.getLogger(__name__)
logger.setLevel(log_level)


In [2]:
import os
import pandas as pd
from openai import OpenAI

from regulations_rag.embeddings import  EmbeddingParameters
from regulations_rag.rerank import RerankAlgos
from regulations_rag.corpus_chat import ChatParameters

import importlib
import cemad_rag.cemad_corpus_index
importlib.reload(cemad_rag.cemad_corpus_index)
from cemad_rag.cemad_corpus_index import CEMADCorpusIndex

import cemad_rag.corpus_chat_cemad
importlib.reload(cemad_rag.corpus_chat_cemad)
from cemad_rag.corpus_chat_cemad import CorpusChatCEMAD


In [3]:
openai_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"),)
chat_parameters = ChatParameters(chat_model = "gpt-4o", temperature = 0, max_tokens = 4096)
#chat_parameters = ChatParameters(chat_model = "gpt-4-turbo", temperature = 0, max_tokens = 500)

embedding_parameters = EmbeddingParameters("text-embedding-3-large", 1024)
key = os.getenv('excon_encryption_key')

corpus_index = CEMADCorpusIndex(key)

rerank_algo  = RerankAlgos.LLM
if rerank_algo == RerankAlgos.LLM:
    rerank_algo.params["openai_client"] = openai_client
    rerank_algo.params["model_to_use"] = chat_parameters.model
    rerank_algo.params["user_type"] = corpus_index.user_type
    rerank_algo.params["corpus_description"] = corpus_index.corpus_description
    rerank_algo.params["final_token_cap"] = 5000 # can go large with the new models


chat = CorpusChatCEMAD(openai_client = openai_client, 
                    embedding_parameters = embedding_parameters, 
                    chat_parameters = chat_parameters, 
                    corpus_index = corpus_index,
                    rerank_algo = RerankAlgos.LLM,   
                    user_name_for_logging = 'test_user')




ANALYSIS:regulations_rag.corpus_chat:test_user: Reset Conversation History


In [4]:
#  May minors purchase foreign exchange
user_content = "Can I use my credit card for online purchases?"
user_content = "Can I open a non-resident account for a CMA resident from another CMA country"
user_content = "What are the foreign currency limits for children?"
user_content = "what do the regulations say about travel?"
user_content = "How much money can an individual take offshore in a year?"

chat.reset_conversation_history()

chat.user_provides_input(user_content)
#chat.user_provides_input(user_content)

ANALYSIS:regulations_rag.corpus_chat:test_user: Reset Conversation History
ANALYSIS:regulations_rag.corpus_chat:test_user question: How much money can an individual take offshore in a year?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
DEV:regulations_rag.corpus_chat:#################   Similarity Search       #################
DEV:regulations_rag.corpus_chat:No relevant workflow found
DEV:regulations_rag.corpus_index:--   No relevant definitions found
DEV:regulations_rag.corpus_index:Selecting the top 15 items based on cosine-similarity score
DEV:regulations_rag.corpus_index:0.2027:                CEMAD:            B.2(B)(i):        question: gAAAAABmtI1JMgtzxh3eyqtETjrag4Py6BQoQcMn4TidPOP7m5TBJn97AZQ6YCe-F8CtYfQpGfvr6Wzm0bgCcsHl0d8NTZjCyo2q9puqHw8yQ1lenJN6ZPkEHpAiPK06HVkZaQ-15CHf
DEV:regulations_rag.corpus_index:0.2190:                CEMAD:               B.4(A):        question: gAAAAABmtI1KTpV3CzVouj7dfBWpEAd5J2mM5FGr4XmBD8obfj1Cht4WBqn25SGlCp

In [5]:
chat.create_openai_assistant_message(chat.messages_intermediate[-1])['content']

'An individual can take offshore up to R1 million per calendar year under the single discretionary allowance.  \nReference:  \nSection B.4(B)(ii) from Currency and Exchange Control Manual for Authorised Dealers  \n'

In [5]:
from IPython.display import Markdown, display

raw_response = chat.messages_intermediate[-1]

llm_reply = raw_response['content']
df_definitions = raw_response['definitions']
df_search_sections = raw_response['sections']



response_dict = chat._check_response(llm_reply, df_definitions, df_search_sections)
if response_dict['path'] == chat.Prefix.ALTERNATIVE.value:
    assistant_response = chat._reformat_assistant_answer(response_dict, df_definitions, df_search_sections)
    row_to_add_to_messages = {"role": "assistant", "content": assistant_response, "section_reference": pd.DataFrame()}
else:
    llm_answer, df_references_list = chat._extract_assistant_answer_and_references(response_dict, df_definitions, df_search_sections)
    row_to_add_to_messages = {"role": "assistant", "content": response_dict['answer'], "section_reference": df_references_list}


# references = chat.references[m]
# for index, row in references.iterrows():
#     display(Markdown(row['text']))    


In [6]:
#display(Markdown(assistant_response))
#assistant_response
response_dict = chat._check_response(llm_reply, df_definitions, df_search_sections)
chat._reformat_assistant_answer(response_dict, df_definitions, df_search_sections)

'An individual can take up to a total amount of R10 million per calendar year offshore as a foreign capital allowance for investment purposes abroad. Additionally, within the single discretionary allowance, an individual can take up to R1 million per calendar year offshore. Therefore, the total amount an individual can take offshore in a year is R11 million.  \nReference:  \nSection B.2(B)(i) from Currency and Exchange Control Manual for Authorised Dealers  \nSection B.4(B)(ii) from Currency and Exchange Control Manual for Authorised Dealers  \n'

In [9]:
for index, row in df_references_list.iterrows():
    my_dict = chat.index.corpus.all_documents
    value_to_find = row["document"]
    # Find the key for the given value
    document_name = next((k for k, v in my_dict.items() if v == value_to_find), None)

    #section_reference = row["section_reference"]



    print(document_name)
    #print(section_reference)
    #print(chat.index.corpus.get_document("CEMAD"))
    #print(chat.index.corpus.get_text("CEMAD", section_reference, add_markdown_decorators=True, add_headings=True, section_only=False))

None
None


In [7]:
#user_content = "What documentation do they require?"
user_content = "What documentation is required?"
chat.user_provides_input(user_content)

# testing = True
# path = "ANSWER:"
# api_response = "Yes, a foreigner can buy property in South Africa. However, if they require local financial assistance for the acquisition, certain conditions apply. Specifically, the 1:1 ratio will apply for financial assistance unless the non-resident is living and working in South Africa, in which case normal lending criteria apply. Additionally, any financial assistance must be secured by an unencumbered Rand deposit or Rand-based asset of equal or higher value, and the facility may not exceed 100% of the Rand value of funds introduced from abroad and invested locally. Facilities for property acquisition cannot be increased based on property revaluation.Reference: 1"
# manual_responses_for_testing = [path + api_response]

# chat.user_provides_input(user_content, testing = testing, manual_responses_for_testing = manual_responses_for_testing)

ANALYSIS:regulations_rag.corpus_chat:test_user question: What documentation is required?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
DEV:regulations_rag.corpus_chat:#################   Similarity Search       #################
INFO:regulations_rag.corpus_chat:Found a potentially relevant workflow: documentation
DEV:regulations_rag.corpus_index:--   No relevant definitions found
DEV:regulations_rag.corpus_index:Selecting the top 15 items based on cosine-similarity score
DEV:regulations_rag.corpus_index:0.3339:                CEMAD:              B.12(A):        question: What documentation is required to conduct a merchanting trade?
DEV:regulations_rag.corpus_index:0.3547:                CEMAD:               B.8(C):        question: What documentation is required for making subsequent payments for a tour?
DEV:regulations_rag.corpus_index:0.3617:                CEMAD:         D.1(C)(i)(d):        question: What documentation is required for import-

In [19]:
llm_reply

'ANSWER: An individual can take up to a total amount of R10 million per calendar year offshore as a foreign capital allowance for investment purposes abroad. Additionally, within the single discretionary allowance, an individual can take up to R1 million per calendar year offshore. Therefore, the total amount an individual can take offshore in a year is R11 million.\n\nReference: 1, 2'

In [7]:
from IPython.display import Markdown, display

# m = chat.corpus.get_text("Forgotten", "1")
# display(Markdown(m))    

d = chat.corpus.get_text("CEMAD", "B.10(A)(i)")
d
#print(chat.corpus.get_text("CovidLocation", "1"))
#print(chat.corpus.get_text("Consent", "7.1.3").strip())
#print(chat.corpus.get_text("GDPR", "8").strip())

'B.10 Insurance and pensions\n\n&nbsp;&nbsp;&nbsp;&nbsp;(A) Introduction\n\n&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(i) Only those branches of the Authorised Dealers referred to in subsection (K) below may effect payments in foreign currency in terms of this section of the Authorised Dealer Manual.'

In [9]:
wf, dfn, section = chat.similarity_search("Can a company use its credit card for online purchases?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
DEV:regulations_rag.corpus_chat:#################   Similarity Search       #################
DEV:regulations_rag.corpus_chat:No relevant workflow found
DEV:regulations_rag.corpus_index:--   No relevant definitions found
DEV:regulations_rag.corpus_index:Selecting the top 15 items based on cosine-similarity score
DEV:regulations_rag.corpus_index:0.2267:                CEMAD:              B.16(E):        question: Can I use my credit card for online purchases?
DEV:regulations_rag.corpus_index:--   Relevant sections found
DEV:regulations_rag.rerank:Re-ranking using LLM
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
DEV:regulations_rag.rerank:--   results requested by LLM filter
DEV:regulations_rag.rerank:B.16(E)


In [8]:
section

,section_reference,text,source,embedding,document,cosine_distance,regulation_text,token_count
0,B.16(E),Can I use my credit card for online purchases?,question,"[-0.06440070271492004, 0.03733660280704498, -0...",CEMAD,0.113945,B.16 Credit and/or debit cards\n (E) Miscel...,279


In [5]:
print(chat._create_system_message(3))

You are answering questions about the General Data Protection Regulation (GDPR) for a Controller based only on the reference extracts provided. You have 3 options:
1) Answer the question. Preface an answer with the tag 'ANSWER:'. All referenced extracts must be quoted at the end of the answer, not in the body, by number, in a comma separated list starting after the keyword 'Reference: '. Do not include the word Extract, only provide the number(s).
2) Request additional documentation. If, in the body of the extract(s) provided, there is a reference to another section that is directly relevant and not already provided, respond with the word 'SECTION:' followed by 'Extract extract_number, Reference section_reference' - for example SECTION: Extract 1, Reference Article 98.
3) State 'NONE:' and nothing else in all other cases



In [ ]:
# BCR
user_content = "What are binding corporate rules?"
user_content = "How do binding corporate rules become approved?"
user_content = "What requirements must binding corporate rules meet?"
user_content = "What details must be spelled out in binding corporate rules?"
user_content = "What rights do individuals have under binding corporate rules?"
user_content = "Who is liable if binding corporate rules are breached?"
user_content = "How are individuals informed about their rights under binding corporate rules?"
user_content = "What is the role of a data protection officer within a group adopting binding corporate rules?"
user_content = "What mechanisms ensure compliance with binding corporate rules?"
user_content = "How is the effectiveness of binding corporate rules verified?"
user_content = "What happens if local laws conflict with the standards of binding corporate rules?"
user_content = "What kind of training is required for employees with access to personal data under binding corporate rules?"
user_content = "What is the process for reporting changes in binding corporate rules?"

# Decision Making
user_content = "What rights do individuals have regarding automated decision-making and profiling?"
user_content = "What does it mean for a decision to have a legal effect on a person?"
user_content = "Under what conditions can an individual be subject to automated decision-making?"
user_content = "What safeguards must be in place when automated decisions are necessary for contracts?"
user_content = "Can special categories of personal data be used in automated decision-making?"

#DPIA
user_content = "When must you conduct a data protection impact assessment?"
user_content = "Who should you consult when carrying out a data protection impact assessment?"
user_content = " What triggers the requirement for a data protection impact assessment?"
user_content = "How do supervisory authorities contribute to the DPIA process?"
user_content = "How do supervisory authorities contribute to the data protection impact assessment process?"
user_content ="What content is required in a data protection impact assessment?"
user_content ="What content is required in a DPIA?"
user_content = "Should individuals' views be considered in the assessment process?"
user_content = "How often should you review the data protection impact assessment?"

#DPO
user_content = "When must you designate a data protection officer?"
user_content = "Can multiple organisations share a DPO?"
user_content = "What criteria must a data protection officer meet?"
user_content = "Can a data protection officer be a contractor?"
user_content = "How should the contact details of the DPO be shared?"
user_content = "What types of processing activities require you to appoint a data protection officer?"
user_content = "How does the scale of data processing affect the requirement to appoint a data protection officer?"
user_content = "What is large scale?"
user_content = "What is the difference between a data protection officer and a controller?"

# intl_transfer
user_content = "What are the conditions for transferring personal data to another country or international organisation?"
user_content = "When is an individual's consent required for data transfer?"
user_content = "How does a contract affect the transfer of personal data?"
user_content = "What constitutes an important reason of public interest for transferring personal data?"
user_content = "When can data be transferred for the establishment, exercise, or defence of legal claims?"
user_content = "Under what circumstances can personal data be transferred to protect someone's vital interests?"
user_content = "What are the rules for transferring data from public registers?"
user_content = "What are the requirements for a non-repetitive transfer based on compelling legitimate interests?"
user_content = "When are public authorities exempt from certain transfer conditions?"
user_content = "How does the law recognise public interest for data transfers?"
user_content = "What documentation is required for assessing data transfers?"

# lead_sa

# data breach
user_content = "What action should you take if a personal data breach occurs?"
user_content = "How soon must you notify the appropriate authority when a data breach is discovered?"
user_content = "Is there any situation where you might not need to notify a supervisory authority about a data breach?"
user_content = "What information must you include when reporting a data breach?"
user_content = "What should be done if all details about a breach can't be reported immediately?"
user_content = "Why is it important to document all personal data breaches and the response to them?"

user_content = "When should you inform individuals about a personal data breach?"
user_content = "What information should be included when notifying an individual about a personal data breach?"
user_content = "Under what conditions can you avoid informing individuals about a personal data breach?"
user_content = "What happens if you don't voluntarily notify individuals about a personal data breach?"


# portability
user_content = "What is the right to data portability?"
user_content = "What does data portability mean for the transfer of data between controllers?"
user_content = "Is the right to data portability unlimited?"

# transparency
user_content = "What information must be provided to individuals when collecting their personal data?"
user_content = "What details about the data controller need to be disclosed during data collection?"
user_content = "What are the required disclosures about the use and processing of personal data?"
user_content = "When is it necessary to inform individuals about the transfer of their personal data to third countries or international organisations?"
user_content = "What rights must individuals be informed about concerning their personal data?"
user_content = "How does the requirement for providing information change if the purpose of data processing changes?"
user_content = "What information must be provided to individuals when their data is collected indirectly?"
user_content = "Within what timeframe must information be provided to the data subject when their data is collected indirectly?"
# user_content = "What actions must be taken if the purpose of data processing changes after initial collection?"
# user_content = "Are there any circumstances under which the obligation to provide information to data subjects does not apply?"
# user_content = "How are data subjects informed about their rights concerning their personal data?"


# Codes
user_content = "Who is encouraged to create codes of conduct for GDPR compliance?"
user_content = "What purposes do codes of conduct serve?"
user_content = "Can codes of conduct address the protection of children's data?"
user_content = "How can codes of conduct assist with personal data breaches?"
user_content = "Can organisations outside the EU adhere to codes of conduct?"
user_content = "What requirements must a code of conduct meet for approval?"
# user_content = "Who approves codes of conduct for local purposes?"
# user_content = "How are codes of conduct evaluated for processing activities across multiple Member States?"
# user_content = "What role does the Commission play in the validity of codes of conduct?"
# user_content = "How is the public informed about approved codes of conduct?"
# user_content = "Where can one find a register of approved codes of conduct?"
# user_content = "Who can monitor compliance with a code of conduct?"
# user_content = "What qualifications must a body meet to monitor compliance with a code of conduct?"
user_content = "How does a body become accredited to monitor a code of conduct?"
# user_content = "What responsibilities does an accredited body have when a code of conduct is breached?"
# user_content = "What actions can an accredited monitoring body take if a code of conduct is infringed?"
# user_content = "Under what circumstances can an accreditation be revoked?"

#user_content = "Can you give me a worfklow for the approval of a code of conduct?"

# territorial scope
user_content = "When does GDPR apply to non-EU entities?"
user_content = "I am not based in the EU. Does GDPR apply to me?"

# video
user_content = "Is video surveillance regulated under gdpr?"

# Covid Health
user_content = "What specific purposes allow healthcare data to be processed without explicit consent?"
user_content = "What are the permissible legal bases for processing health data for scientific research?"

# covid location
user_content = "What types of data should not be collected by contact tracing applications?"

# Concent
user_content = "Can children consent to data collection?"

# Forgotten
user_content = "What is the right to erasure and when does it apply?"


In [1]:
from gdpr_rag.documents.dpo import DPO
from gdpr_rag.documents.consent import Consent
dpo = DPO()
consent = Consent()

In [23]:
file = "./inputs/index/video.parquet"
df = pd.read_parquet(file)
df

,section_reference,text,source,embedding,document
0,2.1,gAAAAABmUH9YDpyn75A6P5UBv2tn3enDzLAPpQYXhXxsuk...,summary,"[-0.05341560021042824, 0.0171151515096426, -0....",Video
1,2.2,gAAAAABmUH9YEm2qVPRmHM-J5GQxVp8K7JCYlxzpbBAt6h...,summary,"[-0.05042532831430435, -0.034690551459789276, ...",Video
2,2.3,gAAAAABmUH9Yt_9fPYROaTi2ZIdE-IGpQpFtdHQ1tI6IpM...,summary,"[-0.05431677773594856, 0.031087687239050865, -...",Video
3,3,gAAAAABmUH9YI-aMX31OE0KoFJtf5__giOrAD1XTEiVt1q...,summary,"[0.011479574255645275, -0.04576420783996582, -...",Video
4,3.1.1,gAAAAABmUH9YMHnezbumcNuKAdDdVAelDbD5cu0VuRpGAz...,summary,"[0.024606412276625633, -0.012803575955331326, ...",Video
...,...,...,...,...,...
123,9.3.2,gAAAAABmUH9YxhYBbjnCjhfyzJSnOqOND6Egd3ErKrbHPX...,question,"[-0.004779711831361055, 0.0069802324287593365,...",Video
124,9.3.2,gAAAAABmUH9YaTg645WLz3LZdFQ_zBII1t0IXvpW96hTbw...,question,"[-0.045810651034116745, 0.048128221184015274, ...",Video
125,9.3.2,gAAAAABmUH9YSxCyP9WC7aFGTLMJSCrcL84pCwxTkSPlhz...,question,"[-0.02516673319041729, -0.024208365008234978, ...",Video
126,10,gAAAAABmUH9YJjSfd9EKsOg7w3xxvlvNGZ5J0T1SWUEaug...,question,"[-0.039777971804142, -0.014415937475860119, -0...",Video
